# Entrenar un robot utilizant NEAT

## Introducció

En aquest notebook, entrenarem un robot perquè seguisca un camí fins a la destinació. Utilitzarem NEAT (NeuroEvolution of Augmenting Topologies) per aconseguir-ho. NEAT és un algorisme d'aprenentatge per reforçament quee utilitza xarxes neuronals per aconseguir el seu objectiu. Aquest algorisme és capaç de crear xarxes neuronals amb topologies molt complexes i eficients.
Partirem de les entrades i sortides de la xarxa neuronal, i l'algorisme s'encarregarà de trobar la millor topologia per aconseguir-ho.

## Preparar les llibreries

Utilitzarem les llibreries d'`aitk` i `neat-python` per aconseguir el nostre objectiu. Començarem instal·lant les llibreries necessàries.

In [ ]:
!pip install aitk neat-python graphviz

In [ ]:
import aitk.robots as bots
from aitk.utils import Grid
from aitk.algorithms import neat

import cv2
import requests
import matplotlib.pyplot as plt

## Neuroevolució

En aquest problema plantejarem les següents definicions:

- **Gen**: Un gen serà un node de la xarxa neuronal.
- **Cromosoma**: Un cromosoma serà una xarxa neuronal amb els pesos corresponents. El seu tamany serà el nombre de pesos de la xarxa neuronal.
- **Població**: Una població serà un conjunt de cromosomes.

El mon i el robot seran els mateixos que en el notebook anterior.

In [ ]:
nom_imatge = "pista_6.png"

url = "https://lawer.github.io/mia/apunts/9.-Rob%C3%B3tica/" + nom_imatge

response = requests.get(url)

# Guardem la imatge

with open(nom_imatge, "wb") as f:
    f.write(response.content)

# Carreguem la imatge en una variable

img = cv2.imread(nom_imatge)

# Mostrem la imatge
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [ ]:
world = bots.World(220, 180, boundary_wall_color="yellow", ground_image_filename=nom_imatge)

amplada_camera = 120
alcada_camera = 50
robot = bots.Scribbler(x=36, y=80, a=90)
robot.add_device(bots.GroundCamera(width=amplada_camera, height=alcada_camera))

world.add_robot(robot)

robot['ground-camera'].watch()
world.watch()

Per calcular el fitness de cada cromosoma, tindrem en compte la distància recorreguda pel robot. Si el robot surt de la matriu, el seu fitness serà 0.

Haurem d'implementar les següents funcions:

- `neat_controller`: Controlador per al robot. Aquí deurem buscar el centre de la imatge captada pel robot i passar-lo a la xarxa neuronal, per calcular el següent moviment i portar-lo a terme.
- `fitness`: Funció per calcular el fitness de cada cromosoma.

Per calcular el calcular el fitness ens poden resultar útils les següents funcions i classes:

- `aitk.utils.Grid`: Classe per a representar una matriu. Li passem el tamany i el mon que ha de monitoritzar. Vejam algunes de les seves funcions:
    - `grid.update(x, y)`: Actualitza l'estat d'una cel·la (per marcar-la com a visitada), 
    - `grid.analyze_visits()`: Calcula el percentatge de cel·les visitades - del 0 si no hi ha cap cel·la visitada, al 1 si ho estan totes.
    - `grid.draw()`: Mostra l'estat actual de la matriu. Ens serà molt útil per veure quines cel·les ha visita  t el robot i quines no i premiar aquells cromosomes que visiten més cel·les.
    - `grid.sequence`: Ens retorna una llista amb les cel·les visitades pel robot. Ens serà útil per calcular el fitness i saber si el robot ha donat la volta a la matriu.
- `robot.stalled`: Funció que ens indica si el robot està contra una paret o no. Ens serà útil per a calcular el fitness, perquè si el robot està aturat, el seu fitness serà 0. També podrem acabar l'entrenament si el robot està aturat.

- `world.time`: Variable que ens indica el temps que ha passat des de l'inici de l'entrenament. Ens serà útil per premiar aquells cromosomes que arriben abans a la destinació. 

In [ ]:
def neat_controller(robot):
    # Controlador per a la xarxa neuronal
    # La xarxa estarà en robot.state["net"]
    # Deurem utilitzar la funció activate de la xarxa per a obtenir les sortides


def fitness(net, seconds=180, real_time=False):
    # Avalua el cromosoma i retorna el seu valor de fitness

    # La xarxa la guardem en el robot per a poder-la utilitzar en el controlador
    robot.state["net"] = net
    # Utilitzem una matriu de 10x10 per a guardar les visites del robot
    robot.state["grid"] = Grid((10, 10), robot.world)


En neat també hem de definir una funció de fitness per tot la població

In [ ]:
def eval_population(pop, config):
    """Aquesta funció rebrà una població de cromosomes i els avaluarà"""
    for genome_id, genome in pop:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        genome.fitness = fitness(net, 180, real_time=False)


Per ultim definirem com volem que funcioni el procés evolutiu

In [ ]:
def run(config_file):
    # Load configuration
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    # Uncomment this to save checkpoints every 5 generations
    # p.add_reporter(neat.Checkpointer(5))

    winner = p.run(eval_population, 20)

    # Display the winning genome.
    print('\nBest genome:\n{!s}'.format(winner))
    node_names = {-1:'d1', -2:'d2', -3:'d3', -4:'timer', 
                  -5:'stall', -6:'bias', 
                  0:'translate', 1: 'rotate'}
    neat.visualize.draw_net(config, winner, True, node_names=node_names)
    neat.visualize.plot_stats(stats, ylog=False, view=True)
    neat.visualize.plot_species(stats, view=True)
    
    return winner

In [ ]:
%%writefile laberint.config
#--- parameters for the Robot Coverage experiment ---#

[NEAT]
fitness_criterion     = max
fitness_threshold     = # Introdueix un valor de fitness que penses que és suficient
pop_size              = # Quan més gran sigui la població, major variabilitat hi haurà a costa de més temps de càlcul
reset_on_extinction   = False

[DefaultGenome]
# node activation options
activation_default      = tanh
activation_mutate_rate  = 0.0
activation_options      = tanh

# node aggregation options
aggregation_default     = sum
aggregation_mutate_rate = 0.0
aggregation_options     = sum

# node bias options
bias_init_mean          = 0.0
bias_init_stdev         = 1.0
bias_max_value          = 30.0
bias_min_value          = -30.0
bias_mutate_power       = 0.5
bias_mutate_rate        = 0.7
bias_replace_rate       = 0.1

# genome compatibility options
compatibility_disjoint_coefficient = 1.0
compatibility_weight_coefficient   = 0.5

# connection add/remove rates
conn_add_prob           = 0.5
conn_delete_prob        = 0.5

# connection enable options
enabled_default         = True
enabled_mutate_rate     = 0.01

feed_forward            = True
initial_connection      = full

# node add/remove rates
node_add_prob           = 0.2
node_delete_prob        = 0.2

# network parameters
num_hidden              = 0 # Es recomanable començar amb 0
num_inputs              = # Introdueix el nombre d'entrades al robot
num_outputs             = # Quants valors de sortida necessitem?

# node response options
response_init_mean      = 1.0
response_init_stdev     = 0.0
response_max_value      = 30.0
response_min_value      = -30.0
response_mutate_power   = 0.0
response_mutate_rate    = 0.0
response_replace_rate   = 0.0

# connection weight options
weight_init_mean        = 0.0
weight_init_stdev       = 1.0
weight_max_value        = 30
weight_min_value        = -30
weight_mutate_power     = 0.5
weight_mutate_rate      = 0.8
weight_replace_rate     = 0.1

[DefaultSpeciesSet]
compatibility_threshold = 3.0

[DefaultStagnation]
species_fitness_func = max
max_stagnation       = 20
species_elitism      = 2

[DefaultReproduction]
elitism            = 2
survival_threshold = 0.2

In [ ]:
winner = run("laberint.config")

Finalment avaluem el millor cromosoma i el posem en marxa

In [ ]:
# Avaluem el guanyador
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         "laberint.config")

winner_net = neat.nn.FeedForwardNetwork.create(winner, config)
fitness(winner_net, 60, real_time=True)

In [ ]:
display("avg_fitness.svg")

El resultat ha segut molt satisfactori, el robot ha aprés a arribar a la destinació en un temps raonable i explorant l'entorn. 

Guardarem el millor cromosoma per poder-lo utilitzar en el futur

In [ ]:
with open("winner.pkl", "wb") as f:    
    import pickle
    
    pickle.dump(winner, f)

Carreguem el model per a comprovar que funciona correctament

In [ ]:
# carreguem el cromosoma guanyador

with open("winner.pkl", "rb") as f:
    import pickle
    winner = pickle.load(f)
    f.close()

# Avaluem el guanyador
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         "laberint.config")
winner_net = neat.nn.FeedForwardNetwork.create(winner, config)
fitness(winner_net, 30, real_time=True)